In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tike
import xdesign as xd

SMALL_SIZE = 8
MEDIUM_SIZE = 8
BIGGER_SIZE = 8

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
NPIXEL = 32
PIXEL_SIZE = 1/NPIXEL
TOTAL_TIME = 1.0
region = np.array([[-PIXEL_SIZE, PIXEL_SIZE], [-1, 1], [-1, 1]])/2
ideal_mean = TOTAL_TIME * PIXEL_SIZE
S_FREQ = 1/2**11


probe = tike.Probe(density_profile=None,
                   width=PIXEL_SIZE,
                   aspect=1.0,
                   lines_per_cm=1/NPIXEL)

def circle_density(x, y, radius=PIXEL_SIZE/2):
    """Return True for (x,y) contained by the circle with given radius"""
    return x**2 + y**2 <= radius**2

def get_mask(A):
    """Return a boolean mask for dimensions 1 & 2 of A"""
    assert A.shape[0] == A.shape[1]
    radius = A.shape[0] / 2.0
    coords = np.arange(A.shape[0]) - A.shape[0] / 2.0 + 0.5,
    x, y = np.meshgrid(coords, coords)
    mask = circle_density(x, y, radius=radius)
    return mask


def get_metric(coverage):
    """Return the standard deviation from the ideal coverage from the region
    inside the cylindrical mask
    """
    ideal_mean = TOTAL_TIME * PIXEL_SIZE
    if coverage.ndim > 3:
        coverage = np.linalg.eigvalsh(coverage)
    x = coverage[:, get_mask(coverage), :].flatten()
    metric = np.sqrt(np.sum((ideal_mean/2 - x)**2) / x.size) - np.mean(x)
    return metric

def fly2D(t, N_rotation=0):
    h = t / TOTAL_TIME - 0.5
    v = 0 * t
    theta = np.pi/3 +  N_rotation * 2 * np.pi * t / TOTAL_TIME
    return theta, h, v

def raster2Dt(t, N_raster=1):
    h = tike.triangle(0.5, N_raster/2, np.pi/2, t)
    v = 0 * t
    theta = np.pi/3 + 2 * np.pi * t / TOTAL_TIME
    return theta, h, v

def raster2Ds(t, N_raster=1):
    h = tike.sinusoid(0.5, N_raster/2, np.pi/2, t)
    v = 0 * t
    theta = np.pi/3 + 2 * np.pi * t / TOTAL_TIME
    return theta, h, v

def semifly(t, N_raster=1):
    h = tike.triangle(0.5, N_raster/2, np.pi/2, t)
    v = 0 * t
    theta = np.pi/3 + tike.staircase(np.pi/N_raster, N_raster, 0, t)
    return theta, h, v

def add_noise(m, dwell, total_time=TOTAL_TIME, total_photons=1.0, noise_percent=0.1):
    m_to_photon = dwell * total_photons / total_time
    p = (m * m_to_photon)
    p1 = p * (1 + noise_percent * np.random.normal(size=m.shape,scale=1))
    m1 = p1 / m_to_photon
    return m1

In [ ]:
np.random.seed(0)
phantom = xd.DogaCircles(n_sizes=4, size_ratio=11/16, n_shuffles=64)
truth = xd.sidebyside(phantom, NPIXEL).T
plt.show()

In [ ]:
mask = get_mask(truth)
plt.imshow(mask)
plt.show()
truth[~mask] = 0

In [ ]:
xprobe = xd.Probe(size=PIXEL_SIZE)
gmin = region[:, 0]
gsize = region[:, 1] - gmin
num_rotations = 2 ** np.arange(2,6)
num_repeats = 1

In [ ]:
gmin, gsize, num_rotations

# FLY2D
More rotations than raster traversals


In [ ]:
# for R in num_rotations:
#     kwargs = {'N_rotation' : R}
#     tstep_guess = TOTAL_TIME / (NPIXEL * max(1, R))
#     procedure = probe.procedure(trajectory=fly2D,
#                               pixel_size=PIXEL_SIZE,
#                               tmin=0, tmax=TOTAL_TIME, tstep=tstep_guess,
#                               tkwargs=kwargs)
#     theta, h, v, dwell = procedure
#     m = xprobe.measure(phantom, theta, h, v)
#     data = [m, theta, h, v, dwell]
#     np.save('./data/doga_grams/fly2D/fly2D_R{:03d}.npy'.format(R), data)

In [ ]:
method = 'fly2D'
for R in num_rotations:
    D = np.load('./data/doga_grams/{0}/{0}_R{1:03d}.npy'.format(method, R))
    m, theta, h, v, dwell = D
    for n in range(num_repeats):
        m1 = add_noise(m, dwell)
        init = np.zeros([NPIXEL, NPIXEL])
        recons = xd.sirt(gmin, gsize, -np.log(m1), theta, h, v, init, niter=5*14, save_interval=5)
        for j in range(len(recons)):
            I = j*5
            np.save('./data/doga_recons/{0}/{0}_R{1:03d}_I{2:03d}_{3:03d}.npy'.format(method, R, I, n), recons[j])

In [ ]:
quality_data = dict()
method = 'fly2D'

nlevels=3
w = [0.0448, 0.2858, 0.3001, 0.2363, 0.1333]

for R in num_rotations:
    I = 0
    iterations = list()
    quality = list()
    std = list()
    for I in range(0, 75, 5):
        repeats = list()
        for k in range(num_repeats):
            recons = np.load('./data/doga_recons/{0}/{0}_R{1:03d}_I{2:03d}_{3:03d}.npy'.format(method, R, I, k))
            recons[~mask] = 0
            iq = xd.ImageQuality(truth, recons)
            iq.compute_quality(nlevels=nlevels)
            repeats.append(np.prod(iq.mets ** (w[0:nlevels] / np.sum(w[0:nlevels]))))
        iterations.append(I)
        quality.append(np.mean(repeats))
        std.append(np.std(repeats))
    quality_data[R] = [iterations, quality, std]

fly2D_quality_data = quality_data

# Semifly

More traversals than rotations


In [ ]:
# for R in num_rotations:
#     kwargs = {'N_raster' : R}
#     tstep_guess = TOTAL_TIME / (NPIXEL * max(1, R))
#     theta = np.array([])
#     h = np.array([])
#     v = np.array([])
#     dwell = np.array([])
#     for i in range(R):
#         procedure = probe.procedure(trajectory=semifly,
#                                   pixel_size=PIXEL_SIZE,
#                                   tmin=i/R*TOTAL_TIME, tmax=(i+1)/R*TOTAL_TIME, tstep=tstep_guess,
#                                   tkwargs=kwargs)
#         thetai, hi, vi, di = procedure
#         theta = np.concatenate([theta, thetai])
#         h = np.concatenate([h, hi])
#         v = np.concatenate([v, vi])
#         dwell = np.concatenate([dwell, di])
#     m = xprobe.measure(phantom, theta, h, v)
#     data = [m, theta, h, v, dwell]
#     np.save('./data/doga_grams/semifly/semifly_R{:03d}.npy'.format(R), data)

In [ ]:
method = 'semifly'
for R in num_rotations:
    D = np.load('./data/doga_grams/{0}/{0}_R{1:03d}.npy'.format(method, R))
    m, theta, h, v, dwell = D
    for n in range(1):
        m1 = add_noise(m, dwell)
        init = np.zeros([NPIXEL, NPIXEL])
        recons = xd.sirt(gmin, gsize, -np.log(m1), theta, h, v, init, niter=5*14, save_interval=5)
        for j in range(len(recons)):
            I = j*5
            np.save('./data/doga_recons/{0}/{0}_R{1:03d}_I{2:03d}_{3:03d}.npy'.format(method, R, I, n), recons[j])

In [ ]:
quality_data = dict()
method = 'semifly'

nlevels=3
w = [0.0448, 0.2858, 0.3001, 0.2363, 0.1333]

for R in num_rotations:
    I = 0
    iterations = list()
    quality = list()
    std = list()
    for I in range(0, 75, 5):
        repeats = list()
        for k in range(num_repeats):
            recons = np.load('./data/doga_recons/{0}/{0}_R{1:03d}_I{2:03d}_{3:03d}.npy'.format(method, R, I, k))
            recons[~mask] = 0
            iq = xd.ImageQuality(truth, recons)
            iq.compute_quality(nlevels=nlevels)
            repeats.append(np.prod(iq.mets ** (w[0:nlevels] / np.sum(w[0:nlevels]))))
        iterations.append(I)
        quality.append(np.mean(repeats))
        std.append(np.std(repeats))
    quality_data[R] = [iterations, quality, std]

semifly_quality_data = quality_data

# Raster2Dt
More traversals than rotations


In [ ]:
# for R in num_rotations:
#     kwargs = {'N_raster' : R}
#     tstep_guess = TOTAL_TIME / (NPIXEL * max(1, R))
#     procedure = probe.procedure(trajectory=raster2Dt,
#                               pixel_size=PIXEL_SIZE,
#                               tmin=0, tmax=TOTAL_TIME, tstep=tstep_guess,
#                               tkwargs=kwargs)
#     theta, h, v, dwell = procedure
#     m = xprobe.measure(phantom, theta, h, v)
#     data = [m, theta, h, v, dwell]
#     np.save('./data/doga_grams/raster2Dt/raster2Dt_R{:03d}.npy'.format(R), data)

In [ ]:
method = 'raster2Dt'
for R in num_rotations:
    D = np.load('./data/doga_grams/{0}/{0}_R{1:03d}.npy'.format(method, R))
    m, theta, h, v, dwell = D
    for n in range(1):
        m1 = add_noise(m, dwell)
        init = np.zeros([NPIXEL, NPIXEL])
        recons = xd.sirt(gmin, gsize, -np.log(m1), theta, h, v, init, niter=5*14, save_interval=5)
        for j in range(len(recons)):
            I = j*5
            np.save('./data/doga_recons/{0}/{0}_R{1:03d}_I{2:03d}_{3:03d}.npy'.format(method, R, I, n), recons[j])

In [ ]:
quality_data = dict()
method = 'raster2Dt'

nlevels=3
w = [0.0448, 0.2858, 0.3001, 0.2363, 0.1333]

for R in num_rotations:
    I = 0
    iterations = list()
    quality = list()
    std = list()
    for I in range(0, 75, 5):
        repeats = list()
        for k in range(num_repeats):
            recons = np.load('./data/doga_recons/{0}/{0}_R{1:03d}_I{2:03d}_{3:03d}.npy'.format(method, R, I, k))
            recons[~mask] = 0
            iq = xd.ImageQuality(truth, recons)
            iq.compute_quality(nlevels=nlevels)
            repeats.append(np.prod(iq.mets ** (w[0:nlevels] / np.sum(w[0:nlevels]))))
        iterations.append(I)
        quality.append(np.mean(repeats))
        std.append(np.std(repeats))
    quality_data[R] = [iterations, quality, std]

raster2Dt_quality_data = quality_data

# Raster2Ds
More traversals than rotations


In [ ]:
# for R in num_rotations:
#     kwargs = {'N_raster' : R}
#     tstep_guess = TOTAL_TIME / (NPIXEL * max(1, R))
#     procedure = probe.procedure(trajectory=raster2Ds,
#                               pixel_size=PIXEL_SIZE,
#                               tmin=0, tmax=TOTAL_TIME, tstep=tstep_guess,
#                               tkwargs=kwargs)
#     theta, h, v, dwell = procedure
#     m = xprobe.measure(phantom, theta, h, v)
#     data = [m, theta, h, v, dwell]
#     np.save('./data/doga_grams/raster2Ds/raster2Ds_R{:03d}.npy'.format(R), data)

In [ ]:
method = 'raster2Ds'
for R in num_rotations:
    D = np.load('./data/doga_grams/{0}/{0}_R{1:03d}.npy'.format(method, R))
    m, theta, h, v, dwell = D
    for n in range(1):
        m1 = add_noise(m, dwell)
        init = np.zeros([NPIXEL, NPIXEL])
        recons = xd.sirt(gmin, gsize, -np.log(m1), theta, h, v, init, niter=5*14, save_interval=5)
        for j in range(len(recons)):
            I = j*5
            np.save('./data/doga_recons/{0}/{0}_R{1:03d}_I{2:03d}_{3:03d}.npy'.format(method, R, I, n), recons[j])

In [ ]:
quality_data = dict()
method = 'raster2Ds'

nlevels=3
w = [0.0448, 0.2858, 0.3001, 0.2363, 0.1333]

for R in num_rotations:
    I = 0
    iterations = list()
    quality = list()
    std = list()
    for I in range(0, 75, 5):
        repeats = list()
        for k in range(num_repeats):
            recons = np.load('./data/doga_recons/{0}/{0}_R{1:03d}_I{2:03d}_{3:03d}.npy'.format(method, R, I, k))
            recons[~mask] = 0
            iq = xd.ImageQuality(truth, recons)
            iq.compute_quality(nlevels=nlevels)
            repeats.append(np.prod(iq.mets ** (w[0:nlevels] / np.sum(w[0:nlevels]))))
        iterations.append(I)
        quality.append(np.mean(repeats))
        std.append(np.std(repeats))
    quality_data[R] = [iterations, quality, std]

raster2Ds_quality_data = quality_data

In [ ]:
R = 1
m, theta, h, v, dwell = np.load('./data/doga_grams/fly2D/fly2D_R{:03d}.npy'.format(R))
m.shape

In [ ]:
m, theta, h, v, dwell = np.load('./data/doga_grams/semifly/semifly_R{:03d}.npy'.format(R))
m.shape

In [ ]:
m, theta, h, v, dwell = np.load('./data/doga_grams/raster2Ds/raster2Ds_R{:03d}.npy'.format(R))
m.shape

In [ ]:
m, theta, h, v, dwell = np.load('./data/doga_grams/raster2Dt/raster2Dt_R{:03d}.npy'.format(R))
m.shape

In [ ]:
# np.save('./data/raster2Ds_qd.npy', raster2Ds_quality_data)
# np.save('./data/fly2D_qd.npy', fly2D_quality_data)
# np.save('./data/raster2Dt_qd.npy', raster2Dt_quality_data)
# np.save('./data/semifly_qd.npy', semifly_quality_data)

In [ ]:
raster2Ds_quality_data = np.load('./data/raster2Ds_qd.npy').item()
fly2D_quality_data = np.load('./data/fly2D_qd.npy').item()
raster2Dt_quality_data = np.load('./data/raster2Dt_qd.npy').item()
semifly_quality_data = np.load('./data/semifly_qd.npy').item()

# Plot quality results combined

In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size': 8})


plt.figure(figsize=(3.3, 2.), dpi=200, frameon=False)
for R in num_rotations:
    
    A = plt.subplot(1,4,1)
    plt.ylabel('MS-SSIM quality index',  fontsize=8)
    iterations, quality, std = semifly_quality_data[R]
    plt.plot(iterations, quality, '-', color=plt.cm.viridis(R/32))
    plt.xlabel('Conventional',  fontsize=8)

    B = plt.subplot(1,4,2, sharey=A, sharex=A)
    iterations, quality, std = fly2D_quality_data[R]
    plt.plot(iterations, quality, '-', color=plt.cm.viridis(R/32))
    plt.setp(B.get_yticklabels(), visible=False)
    plt.xlabel('RAFA', fontsize=8)

    C = plt.subplot(1,4,3, sharey=A, sharex=A)
    iterations, quality, std = raster2Dt_quality_data[R]
    plt.plot(iterations, quality, '-', color=plt.cm.viridis(R/32))
    plt.setp(C.get_yticklabels(), visible=False)
    plt.xlabel('triangular', fontsize=8)

    
    D = plt.subplot(1,4,4, sharey=A, sharex=A)
    iterations, quality, std = raster2Ds_quality_data[R]
    plt.plot(iterations, quality, '-', color=plt.cm.viridis(R/32))
    plt.setp(D.get_yticklabels(), visible=False)
    plt.xlabel('sinusoidal', fontsize=8)
    
plt.suptitle('SIRT reconstruction quality',  fontsize=8)
# plt.legend(num_rotations, loc='upper center')
plt.subplots_adjust(wspace=0.1, top=0.90, bottom=0.2, left=0.15)
plt.savefig('./figures/converge_fly_quality.pdf', dpi=600, pad_inches=0)
plt.savefig('./figures/converge_fly_quality.png', dpi=600, pad_inches=0)
plt.show()

In [ ]:
semifly_quality_data

In [ ]:
plt.figure(figsize=(8.4/2.54, 6.5/2.54), dpi=200, frameon=False)

A = plt.subplot(len(num_rotations), 1, 1)
plt.xlim([0, 1])

for i, R in enumerate(num_rotations):
    plt.subplot(len(num_rotations),1,len(num_rotations)-i, sharex=A)
    max_qual = list()
    std_qual = list()
    iterations, quality, std = semifly_quality_data[R]
    j = np.argmax(quality)
    max_qual.append(quality[j])
    std_qual.append(std[j])
    iterations, quality, std = fly2D_quality_data[R]
    j = np.argmax(quality)
    max_qual.append(quality[j])
    std_qual.append(std[j])
    iterations, quality, std = raster2Dt_quality_data[R]
    j = np.argmax(quality)
    max_qual.append(quality[j])
    std_qual.append(std[j])
    iterations, quality, std = raster2Ds_quality_data[R]
    j = np.argmax(quality)
    max_qual.append(quality[j])
    std_qual.append(std[j])
    
    plt.annotate("{}".format(R), (1.02, 0.5),
                 xycoords='axes fraction', va='center', ha='left')
    
    plt.barh([0,1,2,3], max_qual, xerr=std_qual,
             color=plt.cm.viridis([0, 0.333, 0.666, 1.0]),
             ecolor=plt.cm.viridis([1.0, 1.0, 0, 0]))
    
    plt.gca().set_yticks([0,1,2,3])
    plt.gca().set_yticklabels(['conventional', 'RAFA', 'triangular', 'sinudoidal'])
    if i != 0:
        plt.setp(plt.gca().get_xticklabels(), visible=False)
    else:
        plt.xlabel('Max Reconstruction MS-SSIM Quality')
    
# plt.suptitle('SIRT reconstruction quality',  fontsize=8)
# plt.legend(num_rotations, loc='upper center')
plt.subplots_adjust(hspace=0.25, top=0.99, bottom=0.15, left=0.25)
plt.savefig('./figures/fly_quality.pdf', dpi=600, pad_inches=0)
plt.savefig('./figures/fly_quality.png', dpi=600, pad_inches=0)
plt.show()

In [ ]:
I = 20
plt.figure(figsize=(8.4/2.54, 8.4/2.54), dpi=200, frameon=False)
truth_nan = truth.copy()
truth_nan[~mask] = np.nan

for i in range(len(num_rotations)):
    
    R = num_rotations[i]
    
    A = plt.subplot(len(num_rotations), 4, 4*(len(num_rotations) -1 - i) +1)
#     plt.imshow(truth_nan, origin='lower', cmap=plt.cm.inferno)
#     plt.setp(A.get_xticklabels(), visible=False)
#     # plt.annotate('(a)', (0,0), color='white', va='bottom')
#     plt.axis('off')
    plt.annotate(R, (-0.1, 0.5), xycoords='axes fraction', ha='right', fontsize=8)
#     if i == 0:
#         plt.annotate('truth', (0.5, -0.2), xycoords='axes fraction', ha='center', fontsize=8)
        
#     A = plt.subplot(len(num_rotations),4,4*(5 - i) +1)
    recons = np.load('./data/doga_recons/semifly/semifly_R{:03d}_I{:03d}_000.npy'.format(R, I))
    recons[~mask] = np.nan
#     print(recons)
    plt.imshow(recons, origin='lower', cmap=plt.cm.inferno)
    plt.setp(A.get_yticklabels(), visible=False)
    plt.setp(A.get_xticklabels(), visible=False)
    # plt.annotate('(b)', (0,0), color='white', va='bottom')
    plt.axis('off')
    if i == 0:
        plt.annotate('Conventional', (0.5, -0.2), xycoords='axes fraction', ha='center', fontsize=8)

    B = plt.subplot(len(num_rotations), 4, 4*(len(num_rotations) -1 - i) +2)
    recons = np.load('./data/doga_recons/fly2D/fly2D_R{:03d}_I{:03d}_000.npy'.format(R, I))
    recons[~mask] = np.nan
    plt.imshow(recons, origin='lower', cmap=plt.cm.inferno)
    plt.setp(B.get_yticklabels(), visible=False)
    plt.setp(B.get_xticklabels(), visible=False)
    # plt.annotate('(b)', (0,0), color='white', va='bottom')
    plt.axis('off')
    if i == 0:
        plt.annotate('RAFA', (0.5, -0.2), xycoords='axes fraction', ha='center', fontsize=8)

    C = plt.subplot(len(num_rotations),4,4*(len(num_rotations) -1 - i) +3)
    recons = np.load('./data/doga_recons/raster2Dt/raster2Dt_R{:03d}_I{:03d}_000.npy'.format(R, I))
    recons[~mask] = np.nan
    plt.imshow(recons, origin='lower', cmap=plt.cm.inferno)
    # plt.annotate('(c)', (0,0), color='white', va='bottom')
    plt.axis('off')

    if i == 0:
        plt.annotate('triangular', (0.5, -0.2), xycoords='axes fraction', ha='center', fontsize=8)

    D = plt.subplot(len(num_rotations),4, 4*(len(num_rotations) -1 - i) +4)
    recons = np.load('./data/doga_recons/raster2Ds/raster2Ds_R{:03d}_I{:03d}_000.npy'.format(R, I))
    recons[~mask] = np.nan
    plt.imshow(recons, origin='lower', cmap=plt.cm.inferno)
    plt.setp(D.get_yticklabels(), visible=False)
    # plt.annotate('(d)', (0,0), color='white', va='bottom')
    plt.axis('off')

    if i == 0:
        plt.annotate('sinusoidal', (0.5, -0.2), xycoords='axes fraction', ha='center', fontsize=8)
        
# plt.tight_layout()
plt.subplots_adjust(wspace=0.05, hspace=0.05, top=0.95, bottom=0.05)
plt.savefig('./figures/fly_recon_comparison.pdf', dpi=600, pad_inches=0)
plt.savefig('./figures/fly_recon_comparison.png', dpi=600, pad_inches=0)
plt.show()